In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
sys.path.append(os.path.realpath('..'))

In [2]:
import numpy as np
from scipy.spatial.transform import Rotation

from numpy.typing import ArrayLike
from typing import Union

from tracking_v2.motion import ConstantVelocityModel
from tracking_v2.target import NearConstantVelocityTarget
from tracking_v2.sensor import GeometricSensor

from tracking_v3.frame import Frame

## Point 2D

In [3]:
class Point2D:
    # measurement coordinates are (0,0)
    frame: Frame

    y_stdev: float # Up
    z_stdev: float # Left
    
    def __init__(self, vec: ArrayLike, stdev: ArrayLike = [1, 1]):
        self.frame = Frame([0, 0, 0], vec)
        self.y_stdev, self.z_stdev = stdev

    def __repr__(self) -> str:
        return f"Point2D(frame={self.frame})"

In [4]:
p = Point2D([1, 1, 1])
p

Point2D(frame=Frame(origin=[0, 0, 0], rotation=(45.00, -35.26, 0.00)))

## Track in 3D

Update a 3D track with a 2D measurement. Tracker needs to be initialized with a 3D position.

In [9]:
class Tracker2D:
    """Position + velocity"""
    
    def __init__(self):
        self.x     = np.zeros((6, 1))
        self.P     = np.eye(6)
        self.frame = Frame([0, 0, 0], [1, 0, 0])

        self.H     = np.asarray([[1, 0, 0, 0, 0, 0],
                                 [0, 1, 0, 0, 0, 0],
                                 [0, 0, 1, 0, 0, 0]])
        self.motion = ConstantVelocityModel()
        
        self.v = None
        self.S = None
        self.K = None

    def initialize(self, x: ArrayLike):
        x = np.asarray(x).flatten()
        self.x[:len(x), 0] = x
    
    def predict(self, dt: float):
        F = self.motion.F(dt)
        Q = self.motion.Q(dt)
        
        self.x = F @ self.x
        self.P = F @ self.P @ F.T + Q
    
    def update(self, meas: Point2D):
        # TODO support different origins
        assert np.allclose(meas.frame.origin, self.frame.origin)
        
        # 1. express measurement in the track frame
        to_track = meas.frame.transform(self.frame)

        # z is a unit vector, expressed in the track frame, pointing towards the measurement
        z = to_track.transform_vec([1, 0, 0])

        # normalizing the Forward (x) dimension to track's range places the measurement in
        # the plane perpendicular 
        
        z = z.reshape((3, 1)) # column vector
        self.z = z

        R = np.diag([1, meas.y_stdev, meas.z_stdev])
        R = to_track.transform_cov(R)
        
        # 2. calculate innovation
        self.S = self.H @ self.P @ self.H.T + R
        self.v = z - self.H @ self.x

        # 3. Kalman gain
        self.K = self.P @ self.H.T @ np.linalg.inv(self.S)

        # 4. filtered mean + cov
        self.x = self.x + self.K @ self.v
        self.P = self.P - self.K @ self.S @ self.K.T

        # 5. global frame centered at the new target position
        new_frame = self.frame.frame_from_local_direction(self.x[:3, 0])

        # 6. transforms from the current track frame to the new track frame
        t = self.frame.transform(new_frame)

        # 7. rotate state to the new frame
        self.x = t.transform_vec(self.x).reshape((-1, 1))
        self.P  = t.transform_cov(self.P)

        self.frame = new_frame

In [10]:
t = Tracker3D()
t.initialize([10, 0, 0])
t.predict(1)
t.update(Point3D([11, 0, 0]))

In [11]:
t.x

array([[10.7 ],
       [ 0.  ],
       [ 0.  ],
       [ 0.45],
       [ 0.  ],
       [ 0.  ]])

In [12]:
print_rot(t.frame.rotation)

yaw: 0.00, pitch: 0.00, roll: 0.00


In [13]:
t.frame.to_global_vec(t.x[:3,0])

array([10.7,  0. ,  0. ])

In [14]:
with np.printoptions(precision=2, suppress=True):
    print(t.P)

[[0.7  0.   0.   0.45 0.   0.  ]
 [0.   0.7  0.   0.   0.45 0.  ]
 [0.   0.   0.7  0.   0.   0.45]
 [0.45 0.   0.   1.33 0.   0.  ]
 [0.   0.45 0.   0.   1.33 0.  ]
 [0.   0.   0.45 0.   0.   1.33]]


## Compare with linear Kalman Filter

In [15]:
from tracking_v2.kalman import LinearKalmanFilter

In [16]:
kf = LinearKalmanFilter(ConstantVelocityModel(), [[1, 0, 0, 0, 0, 0],
                                                  [0, 1, 0, 0, 0, 0],
                                                  [0, 0, 1, 0, 0, 0]])
kf.initialize([10, 0, 0], np.eye(3))
kf.predict(1)
kf.calculate_innvovation([11, 0, 0], np.eye(3))
kf.update()

In [17]:
kf.x_hat, kf.P_hat

(array([[10.7 ],
        [ 0.  ],
        [ 0.  ],
        [ 0.45],
        [ 0.  ],
        [ 0.  ]]),
 array([[0.7  , 0.   , 0.   , 0.45 , 0.   , 0.   ],
        [0.   , 0.7  , 0.   , 0.   , 0.45 , 0.   ],
        [0.   , 0.   , 0.7  , 0.   , 0.   , 0.45 ],
        [0.45 , 0.   , 0.   , 1.325, 0.   , 0.   ],
        [0.   , 0.45 , 0.   , 0.   , 1.325, 0.   ],
        [0.   , 0.   , 0.45 , 0.   , 0.   , 1.325]]))

In [18]:
def compare(kf, t, title=None):
    tx = t.frame.to_global_vec(t.x).reshape((-1, 1))
    tP = t.frame.to_global_cov(t.P)

    with np.printoptions(precision=6, suppress=True):
        ans = True

        if title is not None and (not np.allclose(kf.x_hat, tx) or not np.allclose(kf.P_hat, tP)):
            print(title)
        
        if not np.allclose(kf.x_hat, tx):
            print("x_hat != x")
            print(kf.x_hat)
            print(tx)
            print(kf.x_hat - tx)
            ans = False

        if not np.allclose(kf.P_hat, tP):
            print("P_hat != P")
            print(kf.P_hat)
            print(tP)
            print(kf.P_hat - tP)
            ans = False
            
        return ans

In [19]:
def compare_1(x0, z):
    kf = LinearKalmanFilter(ConstantVelocityModel(), [[1, 0, 0, 0, 0, 0],
                                                      [0, 1, 0, 0, 0, 0],
                                                      [0, 0, 1, 0, 0, 0]])
    kf.initialize(x0, np.eye(3))
    kf.predict(1)
    kf.calculate_innvovation(z, np.eye(3))
    kf.update()

    t = Tracker3D()
    t.initialize(x0)
    t.predict(1)
    t.update(Point3D(z))

    compare(kf, t)

compare_1([10, 0, 0], [11, 0, 0])

compare_1([10, 0, 0], [12, 0, 0])

compare_1([10, 10, 0], [11, 11, 0])

compare_1([10, 10, 10], [11, 11, 11])

In [20]:
def compare_2(x0, z1, R1, z2, R2):
    kf = LinearKalmanFilter(ConstantVelocityModel(), [[1, 0, 0, 0, 0, 0],
                                                      [0, 1, 0, 0, 0, 0],
                                                      [0, 0, 1, 0, 0, 0]])
    kf.initialize(x0, np.eye(3))

    t = Tracker3D()
    t.initialize(x0)

    p1 = Point3D(z1, R1)
    
    kf.predict(1)
    kf.calculate_innvovation(z1, p1.frame.to_global_cov(np.diag(R1)))
    kf.update()

    t.predict(1)
    t.update(p1)

    compare(kf, t, "\n*** step 1 ***\n")

    p2 = Point3D(z2, R2)

    kf.predict(1)
    kf.calculate_innvovation(z2, p2.frame.to_global_cov(np.diag(R2)))
    kf.update()

    t.predict(1)
    t.update(p2)

    compare(kf, t, "\n*** step 2 ***\n")

compare_2([10, 0, 0], [11, 0, 0], [1, 1, 1], [12, 0, 0], [1, 1, 1])

compare_2([10, 0, 0], [12, 0, 0], [1, 1, 1], [14, 0, 0], [1, 1, 1])

compare_2([10, 10, 0], [11, 11, 0], [1, 1, 1], [12, 12, 0], [1, 1, 1])

compare_2([10, 10, 10], [11, 11, 11], [1, 1, 1], [12, 12, 12], [1, 1, 1])

compare_2([10, 10, 0], [11, 12, 2], [1, 1, 1], [12, 11, 1], [1, 1, 1])

compare_2([10, 10, 0], [11, 12, 2], [1, 2, 1], [12, 11, 1], [1, 2, 1])

compare_2([10, 10, 10], [11, 12, 13], [1, 2, 3], [16, 17, 20], [6, 2, 4])

## Compare on a long sequence

In [21]:
def compare_n(target = NearConstantVelocityTarget(), R=[1, 2, 3], n=100):
    T = 1
    target.cache(T, n+1)

    # initialize with identity covariance matrix
    kf = LinearKalmanFilter(ConstantVelocityModel(), [[1, 0, 0, 0, 0, 0],
                                                      [0, 1, 0, 0, 0, 0],
                                                      [0, 0, 1, 0, 0, 0]])
    kf.initialize(np.zeros(3), np.eye(3))

    t = Tracker3D()
    t.initialize(np.zeros(3))

    for i in np.arange(1, n+1) * T:
        s = target.true_state(i).flatten()[:3]
        p = Point3D(s, R)
    
        kf.predict(T)
        kf.calculate_innvovation(p.frame.to_global_vec([p.range, 0, 0]),
                                 p.frame.to_global_cov(np.diag(R)))
        kf.update()

        t.predict(T)
        t.update(p)

        assert compare(kf, t, f"\n*** step {i} ***\n")

Exactly the same results in a sequence of 100 updates.

In [22]:
compare_n()

## Debug

In [23]:
x0, z1, R1, z2, R2 = [10, 10, 0], [11, 12, 2], [1, 2, 1], [12, 11, 1], [1, 2, 1]

kf = LinearKalmanFilter(ConstantVelocityModel(), [[1, 0, 0, 0, 0, 0],
                                                  [0, 1, 0, 0, 0, 0],
                                                  [0, 0, 1, 0, 0, 0]])
kf.initialize(x0, np.eye(3))

t = Tracker3D()
t.initialize(x0)

p1 = Point3D(z1, R1)

kf.predict(1)
kf.calculate_innvovation(z1, p1.frame.to_global_cov(np.diag(R1)))
kf.update()

t.predict(1)
t.update(p1)

assert compare(kf, t, "\n*** step 1 ***\n")

In [24]:
kf.predict(1)
t.predict(1)

assert compare(kf, t, "\n*** predict ***\n")

#### Both filters have the same state after the predict step

In [25]:
print(t.frame.to_global_vec(t.x), kf.x_hat.flatten())
assert np.allclose(t.frame.to_global_vec(t.x), kf.x_hat.flatten())
assert np.allclose(t.frame.to_global_cov(t.P), kf.P_hat)

[11.27017417 12.18984035  2.3         0.49702467  0.85689405  0.9       ] [11.27017417 12.18984035  2.3         0.49702467  0.85689405  0.9       ]


#### Are the measurements used for update the same?

In [26]:
p2 = Point3D(z2, R2)
p2

Point3D(range=16.31, frame=Frame(origin=[0, 0, 0], rotation=(42.51, -3.52, -0.00)))

In [27]:
print(p2.frame.to_global_vec([p2.range, 0, 0]), z2)
assert np.allclose(p2.frame.to_global_vec([p2.range, 0, 0]), z2)

[12. 11.  1.] [12, 11, 1]


In [28]:
to_track = p2.frame.transform(t.frame)
z2_in_track_frame = to_track.transform_vec([p2.range, 0, 0])
z2_in_track_frame

array([16.26481509, -1.11859073, -0.45226644])

In [29]:
z2_back_to_global = t.frame.to_global_vec(z2_in_track_frame)
print(z2_back_to_global, z2)
assert np.allclose(z2_back_to_global, z2)

[12. 11.  1.] [12, 11, 1]


In [30]:
print(p2.frame.to_global_cov(np.diag(R2)), t.frame.to_global_cov(to_track.transform_cov(np.diag(R2))))
assert np.allclose(p2.frame.to_global_cov(np.diag(R2)), t.frame.to_global_cov(to_track.transform_cov(np.diag(R2))))

[[ 1.45660377e+00 -4.98113208e-01  7.61041152e-18]
 [-4.98113208e-01  1.54339623e+00  2.34701747e-18]
 [ 7.61041152e-18  2.34701747e-18  1.00000000e+00]] [[ 1.45660377e+00 -4.98113208e-01 -8.69685638e-18]
 [-4.98113208e-01  1.54339623e+00 -2.11276227e-17]
 [ 8.68945841e-19 -2.17732069e-17  1.00000000e+00]]


Both measurements translate to the same global $z$ and $R$

In [31]:
p2 = Point3D(z2, R2)

kf.calculate_innvovation(z2, p2.frame.to_global_cov(np.diag(R2)))
kf.update()
t.update(p2)

compare(kf, t, "\n*** update ***\n")

True

In [32]:
tx = t.frame.to_global_vec(t.x).reshape((-1, 1))
tP = t.frame.to_global_cov(t.P)

kf.x_hat, tx

(array([[11.75259112],
        [11.36888014],
        [ 1.30528376],
        [ 0.78484946],
        [ 0.32861146],
        [ 0.20547945]]),
 array([[11.75259112],
        [11.36888014],
        [ 1.30528376],
        [ 0.78484946],
        [ 0.32861146],
        [ 0.20547945]]))

In [33]:
kf.P_hat, tP

(array([[ 1.04427210e+00, -2.97511710e-01,  5.47737016e-18,
          6.76862658e-01, -1.59822150e-01,  2.33685680e-18],
        [-2.97511710e-01,  1.08289916e+00,  2.28530206e-18,
         -1.56531588e-01,  7.09162904e-01,  5.91362648e-19],
        [ 5.47737016e-18,  2.28530206e-18,  7.65166341e-01,
          2.28831963e-18,  6.23925018e-19,  5.34246575e-01],
        [ 6.76862658e-01, -1.56531588e-01,  2.28831963e-18,
          1.21629350e+00, -1.15826355e-01,  2.41367827e-18],
        [-1.59822150e-01,  7.09162904e-01,  6.23925018e-19,
         -1.15826355e-01,  1.23560122e+00,  1.14453357e-18],
        [ 2.33685680e-18,  5.91362648e-19,  5.34246575e-01,
          2.41367827e-18,  1.14453357e-18,  1.10958904e+00]]),
 array([[ 1.04427210e+00, -2.97511710e-01,  2.32843721e-17,
          6.76862658e-01, -1.59822150e-01,  4.24259133e-17],
        [-2.97511710e-01,  1.08289916e+00,  1.39049950e-17,
         -1.56531588e-01,  7.09162904e-01,  5.75183603e-17],
        [ 1.75435399e-17,  1.4

## Debug

In [34]:
np.set_printoptions(precision=2, suppress=True)

In [35]:
frame = Frame([0, 0, 0], [1, 0, 0])
H     = np.asarray([[1, 0, 0, 0, 0, 0],
                    [0, 1, 0, 0, 0, 0],
                    [0, 0, 1, 0, 0, 0]])
motion = ConstantVelocityModel()

P = np.eye(6)
x = np.array([[10, 10, 0, 0, 0, 0]]).T

meas = Point3D([11, 11, 0], [1, 2, 1])

In [36]:
dt = 1

F = motion.F(dt)
Q = motion.Q(dt)

x = F @ x
P = F @ P @ F.T + Q

x, P

(array([[10],
        [10],
        [ 0],
        [ 0],
        [ 0],
        [ 0]]),
 array([[2.33, 0.  , 0.  , 1.5 , 0.  , 0.  ],
        [0.  , 2.33, 0.  , 0.  , 1.5 , 0.  ],
        [0.  , 0.  , 2.33, 0.  , 0.  , 1.5 ],
        [1.5 , 0.  , 0.  , 2.  , 0.  , 0.  ],
        [0.  , 1.5 , 0.  , 0.  , 2.  , 0.  ],
        [0.  , 0.  , 1.5 , 0.  , 0.  , 2.  ]]))

In [37]:
# 1. express measurement in the track frame
to_track = meas.frame.transform(frame)
to_track

Transform(translation=[0.0, 0.0, 0.0], rotation=(45.00, 0.00, 0.00))

In [38]:
# delta_frame transforms from measurement frame to track frame;
# z starts in measurement frame (range-only) and ends in the track frame
z = np.asarray([meas.range, 0, 0])
z = to_track.transform_vec(z)

z = z.reshape((3, 1)) # column vector

R = np.diag([meas.x_stdev, meas.y_stdev, meas.z_stdev])
R = to_track.transform_cov(R)

z, R

(array([[11.],
        [11.],
        [ 0.]]),
 array([[ 1.5, -0.5,  0. ],
        [-0.5,  1.5,  0. ],
        [ 0. ,  0. ,  1. ]]))

In [39]:
# 2. calculate innovation
S = H @ P @ H.T + R
v = z - H @ x

v, S

(array([[1.],
        [1.],
        [0.]]),
 array([[ 3.83, -0.5 ,  0.  ],
        [-0.5 ,  3.83,  0.  ],
        [ 0.  ,  0.  ,  3.33]]))

In [40]:
# 3. Kalman gain
K = P @ H.T @ np.linalg.inv(S)
K

array([[0.62, 0.08, 0.  ],
       [0.08, 0.62, 0.  ],
       [0.  , 0.  , 0.7 ],
       [0.4 , 0.05, 0.  ],
       [0.05, 0.4 , 0.  ],
       [0.  , 0.  , 0.45]])

In [41]:
# 4. filtered mean + cov
x = x + K @ v
P = P - K @ S @ K.T

x, P

(array([[10.7 ],
        [10.7 ],
        [ 0.  ],
        [ 0.45],
        [ 0.45],
        [ 0.  ]]),
 array([[ 0.89, -0.19,  0.  ,  0.57, -0.12,  0.  ],
        [-0.19,  0.89,  0.  , -0.12,  0.57,  0.  ],
        [ 0.  ,  0.  ,  0.7 ,  0.  ,  0.  ,  0.45],
        [ 0.57, -0.12,  0.  ,  1.4 , -0.08,  0.  ],
        [-0.12,  0.57,  0.  , -0.08,  1.4 ,  0.  ],
        [ 0.  ,  0.  ,  0.45,  0.  ,  0.  ,  1.33]]))

In [42]:
# 5. global frame centered at the new target position
new_frame = frame.frame_from_local_direction(x[:3, 0])

# 6. transforms from the current track frame to the new track frame
t = frame.transform(new_frame)

# 7. rotate state to the new frame
x = t.transform_vec(x).reshape((-1, 1))
P = t.transform_cov(P)

x, P

(array([[15.13],
        [-0.  ],
        [ 0.  ],
        [ 0.64],
        [ 0.  ],
        [ 0.  ]]),
 array([[ 0.7 ,  0.  ,  0.  ,  0.45,  0.  ,  0.  ],
        [ 0.  ,  1.08,  0.  , -0.  ,  0.69,  0.  ],
        [ 0.  ,  0.  ,  0.7 ,  0.  ,  0.  ,  0.45],
        [ 0.45,  0.  ,  0.  ,  1.32,  0.  ,  0.  ],
        [-0.  ,  0.69,  0.  , -0.  ,  1.48,  0.  ],
        [ 0.  ,  0.  ,  0.45,  0.  ,  0.  ,  1.33]]))